In [1]:
from chain import Chain, identity, show, foldl, foldr

In [2]:
chain = 1 > Chain.do(identity) >> (lambda x: x+1) & print | (identity, identity)
chain

Chain(1 | identity >== λx -> x+1 >> print <| [identity, identity])

In [3]:
def add(a,b):
    return a+b

Chain.do(show).map(lambda x: x+1, 
                   lambda x: x*2, 
                   lambda x: x/3)\
                .then(foldl(lambda a,b: a+b))

Chain(<empty> | show <| [λx -> x+1, λx -> x*2, λx -> x/3] >== foldl(λa,b -> a+b))

In [2]:
def add(a):
    def add(b):
        return a+b
    return add

def mul(a):
    def mul(b):
        return a*b
    return mul

def div(a):
    def div(b):
        return a/b
    return div

chain1 = Chain.do(identity).map(add(1), mul(2), div(3))
print(chain1)

def process(t: tuple):
    return (t[0], t[0]*2, t[1], t[2]*6, t[2])


chain2 = chain1.do(process)
print(chain2)

chainx = lambda x: Chain.do(chain1.start(x).compute().result).then(chain2.start).then(lambda x: x.compute().result()).start(x).compute().result()

Chain(<empty> | identity <| [add, mul, div])
Chain(<empty> | process)


In [11]:
Chain.do(identity).glue(chain1).glue(chain2)

Chain(<empty> | identity >== identity <| [add, mul, div] >== process)

In [10]:
r = chainx(1)

In [11]:
r

(2, 4, 2, 18.0, 3.0)

In [15]:
def add_one(x):
            return x + 1

chain1 = Chain.do(lambda x: x).glue(Chain.do(add_one)).start(1).compute().result()
#chain2 = Chain.do(add_one).compute().start(1).result()

In [5]:
def printx(msg):
    def _print(*args, **kwargs):
        print(msg)
    _print.__name__ = "print"
    return _print

def format_result(t):
    name, last = t
    return f"Hello, {name} {last}!"

In [6]:
(Chain.do(printx("What's your first name?"), action='run')
      .then(input)
      .run(printx("What's your last name?"))
      .map(identity, input)
      .then(format_result)
      .run(print))

Chain(<empty> | print >== raw_input >> print <| [identity, raw_input] >== format_result >> print)

In [23]:
chain = (... > 
         (Chain.do(printx("What's your first name?"), action='run') 
          >> input 
          & printx("What's your last name?") 
          | (identity, input)) 
          >> format_result 
          & print)

In [24]:
chain.compute()

What's your first name?
What's your last name?
Hello, Joan Gonzalez!


Chain(Hello, Joan Gonzalez! | <empty>)

In [26]:
def add(a,b):
    return a+b

Chain.do(show).map(lambda x: x+1, 
                   lambda x: x*2, 
                   lambda x: x/3)\
                .then(foldl(lambda a,b: a+b))

Chain(<empty> | show <| [λx -> x+1, λx -> x*2, λx -> x/3] >== foldl(λa,b -> a+b)))

In [29]:
((Chain.do(show) | (lambda x: x+1, 
                  lambda x: x*2, 
                  lambda x: x/3))
                >> foldl(lambda a,b: a+b)
)

Chain(<empty> | show <| [λx -> x+1, λx -> x*2, λx -> x/3)] >== foldl(λa,b -> a+b))

In [122]:
import ast

def Try(f):
    try:
        return f()
    except Exception as e:
        return e


def longest_lambda_substring(s):
    longest_lambda = ''
    for i in range(len(s), -1, -1):
        for j in range(0, len(s) - i + 1):
            substring = s[j:j+i]
            result = Try(lambda: ast.parse(substring, mode='eval'))
            
            if isinstance(result, Exception):
                continue

            if isinstance(result.body, ast.Lambda) and len(substring) > len(longest_lambda):
                longest_lambda = substring
    return longest_lambda

In [125]:
longest_lambda_substring('lambda x: (x,x)')

'lambda x: (x,x)'

In [4]:
(Chain.do(identity).map(lambda x: x+1,
                        lambda x: x*2,
                        lambda x: x/3)
                    .mapply(lambda x: x+100,
                            lambda x: x*20,
                            lambda x: x/30)
                    .foreach(lambda x: x >= 40)
                    .start(1))

Chain(1 | identity <| [λx -> x >= 40), λx -> x*2, λx -> x/3] || [λx -> x+100, λx -> x*20, λx -> x/30] <$> λx -> x >= 40)